In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
#from scipy import fft
from scipy import signal as sig
from scipy.fft import fft, fftfreq
from IPython.display import Audio
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import math
import wave
import numpy as np
import matplotlib.pyplot as plt
import sys
import os
import librosa, librosa.display
import IPython.display as ipd
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
import scipy.io.wavfile as wavfile
import scipy
import scipy.fftpack
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import plotly.graph_objs as go
import warnings
warnings.simplefilter('ignore')

In [2]:
#data = pd.read_csv(r'C:\Users\DELL\COV_Project\BTP_2\Frequency Characteristics\Coswara\data.csv')
data = pd.read_csv(r'../input/coswara-dtale-dataset/data.csv')

In [3]:
!pip install dtale

     |████████████████████████████████| 11.4 MB 4.9 MB/s            
     |████████████████████████████████| 9.5 MB 40.2 MB/s            
     |████████████████████████████████| 96 kB 3.3 MB/s             
     |████████████████████████████████| 242 kB 68.8 MB/s            
     |████████████████████████████████| 45 kB 2.2 MB/s             
     |████████████████████████████████| 209 kB 48.3 MB/s            
     |████████████████████████████████| 22.3 MB 41.9 MB/s            
     |████████████████████████████████| 1.2 MB 28.2 MB/s            
     |████████████████████████████████| 8.5 MB 44.9 MB/s            
     |████████████████████████████████| 40 kB 3.6 MB/s             
     |████████████████████████████████| 79.9 MB 55.1 MB/s            
     |████████████████████████████████| 642 kB 39.6 MB/s            
  Preparing metadata (setup.py) ... - done
     |████████████████████████████████| 47 kB 2.5 MB/s             
  Preparing metadata (setup.py) ... - done
     |█████████

In [4]:
cols = ['ID', 'Age', 'Status', 'Record_date',
       'gender', 'Country', 'Locality', 'State',
       'Smoker', 'Cold', 'Hypertension', 'Diabetes', 'Cough', 'Diarrheoa', 'Fever', 'Loss of Smell & Taste (True/False)',
       'Muscle Pain (True/False)', 'COVID_status', 'Using Mask (y/n)',
       'Vaccination status (y->both doses, p->one dose(partially vaccinated), n->no doses)',
       'Breathing Difficulties (True/False)',
       'Respiratory illnesses other than the listed ones (True/False)',
       'Fatigue (True/False)', 'Sore_Throat',
       'Ischemic Heart Disease (True/False)', 'Asthma (True/False)',
       'Pre-existing conditions other than the listed ones (True/False)',
       'Chronic_Lung_Disease', 'Pneumonia', '(0,20]', '(20,40]', '(40,60]',
       '(60,80]', '(80,100]', '(100,120]', '(120,140]', '(140,160]',
       '(160,180]', '(180,200]', '(200,220]', '(220,240]', '(240,260]',
       '(260,280]', '(280,300]', '(300,320]', '(320,340]', '(340,360]',
       '(360,380]', '(380,400]', '(400,420]', '(420,440]', '(440,460]',
       '(460,480]', '(480,500]', '(500,520]', '(520,540]', '(540,560]',
       '(560,580]', '(580,600]', '(600,620]', '(620,640]', '(640,660]',
       '(660,680]', '(680,700]', '(700,720]', '(720,740]', '(740,760]',
       '(760,780]', '(780,800]', '(800,820]', '(820,840]', '(840,860]',
       '(860,880]', '(880,900]', '(900,920]', '(920,940]', '(940,960]',
       '(960,980]', '(980,1000]', 'source']
data = data[cols]

In [5]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

options = ['male', 'female']
data = data[data['gender'].isin(options)] 
status = ['p', 'n']
data = data[data['COVID_status'].isin(status)] 
f_data = data

In [6]:
data.head()

,ID,Age,Status,Record_date,gender,Country,Locality,State,Smoker,Cold,Hypertension,Diabetes,Cough,Diarrheoa,Fever,Loss of Smell & Taste (True/False),Muscle Pain (True/False),COVID_status,Using Mask (y/n),"Vaccination status (y->both doses, p->one dose(partially vaccinated), n->no doses)",Breathing Difficulties (True/False),Respiratory illnesses other than the listed ones (True/False),Fatigue (True/False),Sore_Throat,Ischemic Heart Disease (True/False),Asthma (True/False),Pre-existing conditions other than the listed ones (True/False),Chronic_Lung_Disease,Pneumonia,"(0,20]","(20,40]","(40,60]","(60,80]","(80,100]","(100,120]","(120,140]","(140,160]","(160,180]","(180,200]","(200,220]","(220,240]","(240,260]","(260,280]","(280,300]","(300,320]","(320,340]","(340,360]","(360,380]","(380,400]","(400,420]","(420,440]","(440,460]","(460,480]","(480,500]","(500,520]","(520,540]","(540,560]","(560,580]","(580,600]","(600,620]","(620,640]","(640,660]","(660,680]","(680,700]","(700,720]","(720,740]","(740,760]","(760,780]","(780,800]","(800,820]","(820,840]","(840,860]","(860,880]","(880,900]","(900,920]","(920,940]","(940,960]","(960,980]","(980,1000]",source
0,9hXEs9OejdVxG6JJGCyKQpqVvy43,67,positive_moderate,20210719,male,India,Annur coimbator,Tamil Nadu,n,True,True,True,True,True,True,True,True,p,y,n,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,122462.20,110512.60,122038.70,165540.50,187686.50,234166.10,464990.6,1497269.0,4272285.0,6670527.0,6813176.0,7407627.0,5660090.0,7203625.0,10155080.0,8436202.0,5658043.0,2209144.0,2278392.0,1815391.0,787478.7,1155470.0,1920140.0,430031.7,381051.6,419583.0,1339097.0,1779969.0,3653937.0,3988582.0,2493865.0,2427367.0,1405778.0,580211.2,1024302.0,1169316.0,1471690.0,1479020.0,1526700.0,1655924.0,811980.9,543695.2,615953.3,880054.9,770533.1,1.938898e+05,1.943286e+05,1.750968e+05,3.804597e+05,2.014790e+05,../input/correct-directory-extraction/cough_outputs/9hXEs9OejdVxG6JJGCyKQpqVvy43/cough-shallow.wav
1,dPi1G2i821MDlbBUQWboTaExz023,24,healthy,20210816,male,India,Coimbatore,Tamil Nadu,n,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,n,y,y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,592186.50,765560.90,1176545.00,3394010.00,6056102.00,9178112.00,8054089.0,10818960.0,14923850.0,16215510.0,17957590.0,13504610.0,12587180.0,11845560.0,11671690.0,11583550.0,11329310.0,12382920.0,12156820.0,10602480.0,9186106.0,8328820.0,8118411.0,6559482.0,6876317.0,7129267.0,6856931.0,7466433.0,6866753.0,8820287.0,8991361.0,8519854.0,10095220.0,11077650.0,8958233.0,7698331.0,5565252.0,4931148.0,4414184.0,4509145.0,4584179.0,5479105.0,5814686.0,5318021.0,5222335.0,2.284924e+06,2.251588e+06,2.259810e+06,2.213096e+06,2.142489e+06,../input/correct-directory-extraction/cough_outputs/dPi1G2i821MDlbBUQWboTaExz023/cough-shallow.wav
4,OPOiHCtB3WXhEdnsofhMpnP8bak1,38,recovered_full,20210715,male,India,Coimbatore,Tamil Nadu,n,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,p,n,p,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14588.07,16030.26,21006.19,37856.09,50858.29,70833.62,449869.4,2230532.0,3053779.0,1890753.0,1372765.0,2665555.0,2431799.0,1326064.0,887358.4,531423.2,297489.3,681464.9,1276455.0,1191128.0,1177923.0,938598.7,570374.0,485734.5,660542.2,421256.9,608421.1,738819.4,522462.2,739137.4,709124.1,482477.8,364746.8,438868.4,541268.3,827763.8,720395.8,436168.4,440096.6,1199676.0,1689456.0,1339581.0,789055.7,504429.1,272651.0,1.401376e+05,1.673620e+05,1.221615e+05,9.308663e+04,1.103769e+05,../input/correct-directory-extraction/cough_outputs/OPOiHCtB3WXhEdnsofhMpnP8bak1/cough-shallow.wav
5,XbOUJCUl8GWEpQpIRMvujDE1sTE2,57,positive_moderate,20210717,female,India,Coimbatore,Tamil Nadu,n,True,True,True,True,True,True,True,True,p,y,n,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,111957.50,139734.90,119548.60,123295.20,151958.00,217197.00,494058.4,1811326.0,3693804.0,7100896.0,6047195.0,5467023.0,7702567.0,10462510.0,7409737.0,2767386.0,5350628.0,6791030.0,6899710.0,3782310.0,2798111.0,5382775.0,5009663.0,6858340.0,3356170.0,1094191.0,412427.5,718032.0,938543.2,1951414.0,3717743.0,4644502.0,201254

### Number of Missing Values

In [7]:
a = data.isnull().mean() * 100
a = a.to_frame()
sorted_df = a.sort_values([0], ascending=False)
sorted_df.rename({0: '% of missing valus'}, axis=1, inplace=True)
sorted_df.head(20)

,% of missing valus
Ischemic Heart Disease (True/False),97.764531
Chronic_Lung_Disease,95.827124
Pneumonia,94.932936
Diarrheoa,94.336811
Asthma (True/False),93.740686
Respiratory illnesses other than the listed ones (True/False),92.548435
Pre-existing conditions other than the listed ones (True/False),89.120715
Hypertension,87.034277
Diabetes,84.202683
Loss of Smell & Taste (True/False),82.712370


# Use this for getting the DTale table

In [8]:
# import dtale
# import dtale.app as dtale_app

# #dtale_app.USE_NGROK = True

# d=dtale.show(data)

# d.main_url()

In [9]:
data['Chronic_Lung_Disease'].value_counts()

True    28
Name: Chronic_Lung_Disease, dtype: int64